# Criando features

- Esse notebook é a continuação do [1. EDA](1.%20EDA.ipynb)

# Ìndice
1. [Resumo e objetivo](#Resumo-e-Objetivo)
- [Features Básicas](#Features-Básicas)
    - [Feature shop_name](#Features-shop_name)
    - [Feature category_id](#Features-category_id)
    - [Retirando item_name e date](#Retirando-item_name-e-date)
- [Criando Features Novas](#Criando-Features-Novas)
    - [Revenue](#Revenue)
    - [Semestre e ano](#Semestre-e-ano)  
    - [Médias mensais](#Médias-mensais)
        - [Média por loja](#Média-por-loja)
        -  [Média por item](#Média-por-item)
        -  [Média dos itens por loja](#Média-dos-itens-por-loja)
    - [Lagged Variables](#Lagged-Variables) (Em construção)
    - [Latitude e Longitude](#Latitude-e-Longitude)
    - [Unindo os dataframes](#unindo)
- [Apêndice](#Apêndice)
- [Referências](#Referências)

### Coisas a serem feitas:
- [ ] **Comentar o código e deixar as variáveis mais fáceis de serem lidas.**
- Colocar as outras featues.
    - [ ] Lagged. [1](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/) [2](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/)
    - [x] Meses.  
    - [ ] Médias.
        - [ ] Fazer a média do item_cnt por loja por item.
        - [ ] Fazer a média do item_cnt por categória.
- [ ] Tem que melhorar muito a explicação das laggeds.
- [ ] Unir os dataframes.

# Carregando os arquivos

In [83]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

path = 'data'

train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')
geo = pd.read_csv(path + '/geo_shop.csv')

# Resumo e objetivo

Neste notebook nos vamos criar várias features novas e depois vamos voltar em [1. EDA](1.%20EDA.ipynb) e tentar extrair alguma informação.

# Features Básicas

- [Feature shop_name](#Features-shop_name)
- [Feature category_id](#Features-category_id)
- [Retirando item_name e date](#Retirando-item_name-e-date)

### Features shop_name

A feature categórica shop_name, também possui informação sobre localização e tipos de lojas.

- Адыгея ТЦ "Мега"         => Adygea shopping center "Mega"	
- Волжский ТЦ "Волга Молл" => Volzhsky shopping center "Volga Mall"
- Вологда ТРЦ "Мармелад"   => Shopping center Vologda "Marmalade"	
- Якутск ТЦ "Центральный"  => Centro comercial Yakutsk "Central"	
- Ярославль ТЦ "Альтаир"   => Centro comercial Yaroslavl "Altair"

In [84]:
# Corrigindo uma entrada.
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

# Separando as palavras (split) e escolhendo a primeira (map(lambda x: x[0])).
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])

#Corringindo um nome.
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

print('Nome das cidades:\n',shops['city'].head())

Nome das cidades:
 0      Якутск
1      Якутск
2      Адыгея
3    Балашиха
4    Волжский
Name: city, dtype: object


> ### Obs.:
> Note que é possível retirar mais informações dos nomes, como o tipo de estabelecimento (Shopping, Centro comercial, e etc), o tipo do shopping (Volga Mall) e até mesmo a localização na cidade (Central).

Apesar de termos as localizações separadas, a maioria dos algoritmos só enchergam números, sendo o XGboost uma excessão. Então, temos que rotular cada nome de cidade com um número.

<a name="back1"></a> Deixei um exemplo simples [aqui](#go1) de como fazer isso para quem ainda está começando.

In [85]:
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops.head()

,shop_name,shop_id,city,city_code
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,29
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,29
2,"Адыгея ТЦ ""Мега""",2,Адыгея,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,1
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,2


In [86]:
shops.drop(['shop_name'], axis='columns', inplace=True)
shops.head()

,shop_id,city,city_code
0,0,Якутск,29
1,1,Якутск,29
2,2,Адыгея,0
3,3,Балашиха,1
4,4,Волжский,2


> Ainda vou usar os nomes da cidade, só por isso não retirei do dataset.

### Latitude e Longitude

No final do notebook [1. EDA](1.%20EDA.ipynb) eu queria colocar alguns mapas com as localizações e vendas das lojas, mas acabei achando interessante incluir também a latitude e longitude como features.
Para criar o arquivo com as latitudes e longitudes eu usei a feature com o nome das cidades e esse [site](https://www.latlong.net/).

In [87]:
geo.head(8)

,name,name_PT-BR,latitude,longitude
0,Адыгея,Adygea,44.488950,39.730820
1,Балашиха,Balashikha,55.797040,37.934500
2,Волжский,Volzhsky,48.786300,44.751570
3,Вологда,Vologda,59.218067,39.897804
4,Воронеж,Voronezh,51.675495,39.208881
5,Выездная,Sair,45.157089,42.966549
6,Жуковский,Zhukovsky,55.577942,38.125069
7,Интернет-магазин,Loja Online,NaN,NaN


O problema é como substituir os NaN para lojas online. Se colocarmos um valor $(0,0)$, acredito eu, que vai ser como adicionar um outlier, o que não é bom para algoritmos de previsão. Possibilidades:
   - Tirar a média das latitudes e longitudes.
   - Usar algum algoritmo igual ao Knn.
   
Independente da opção escolhida, temos que adicionar um marcador para dizer quais lojas são online e quais são físicas.

In [88]:
geo['is_online'] = 0

mask = geo['latitude'].isna()

geo.loc[mask, 'is_online'] = 1

del mask

In [89]:
geo.fillna(value={'latitude':geo['latitude'].mean() ,'longitude':geo['longitude'].mean()}, inplace=True)
geo.head(8)

,name,name_PT-BR,latitude,longitude,is_online
0,Адыгея,Adygea,44.488950,39.730820,0
1,Балашиха,Balashikha,55.797040,37.934500,0
2,Волжский,Volzhsky,48.786300,44.751570,0
3,Вологда,Vologda,59.218067,39.897804,0
4,Воронеж,Voronezh,51.675495,39.208881,0
5,Выездная,Sair,45.157089,42.966549,0
6,Жуковский,Zhukovsky,55.577942,38.125069,0
7,Интернет-магазин,Loja Online,54.868193,51.085524,1


In [90]:
shops = pd.merge(shops, geo, right_on=['name'], left_on=['city'], how='left')
shops.drop(['city', 'name', 'name_PT-BR'], axis=1, inplace=True)
shops.head()

,shop_id,city_code,latitude,longitude,is_online
0,0,29,62.035454,129.675476,0
1,1,29,62.035454,129.675476,0
2,2,0,44.488950,39.730820,0
3,3,1,55.797040,37.934500,0
4,4,2,48.786300,44.751570,0


### Feature category_id

O mesmo procedimento vai ser realizado para os nomes das categorias. Vamos dividi-las em duas features, tipo e subtipo. Exemplo:
    - Аксессуары - PS2 => Tipo: Acessórios / Subtipo: PS2.
    
Da mesma forma iremos definir um número para cada um dos tipos e subtipos.

In [91]:
# Separando as duas categorias.
items_cat['split'] = items_cat['item_category_name'].str.split('-')

# Criando uma coluna só com o tipo e depois criando os rótulos.
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])

# Criando uma coluna só com o subtipo e depois criando os rótulos.
items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])

items_cat.head()

,item_category_name,item_category_id,split,type,type_code,subtype,subtype_code
0,PC - Гарнитуры/Наушники,0,"[PC , Гарнитуры/Наушники]",PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,"[Аксессуары , PS2]",Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,"[Аксессуары , PS3]",Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,"[Аксессуары , PS4]",Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,"[Аксессуары , PSP]",Аксессуары,1,PSP,13


In [92]:
# Retirando colunas que não tem mais utilidade.
items_cat.drop(['item_category_name', 'split', 'type', 'subtype'], axis='columns', inplace=True)
items_cat.head()

,item_category_id,type_code,subtype_code
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


### Retirando item_name e date

 Vou retirar duas colunas do banco de dado, item_name e date. As datas não vão ser precisas porque vou utilizar somente os meses e para isso já temos a coluna date_block_num. No nome dos itens deve ter alguma informação interessante, mas vai ser muito complicado separar essas coisas.

In [93]:
items.drop(['item_name'], axis=1, inplace=True)

In [94]:
train.drop(['date'], axis='columns', inplace=True)

# Criando Features Novas

Criar features é tão importante quanto fazer uma boa EDA. Algumas features podem fazer a diferença para modelo preditivos. Vou abordar a construção das seguintes features:

   - [Revenue](#Revenue)
   - [Semestre e ano](#Semestre-e-ano)  
   - [Médias mensais](#Médias-mensais)
       - [Média por loja](#Média-por-loja)
       -  [Média por item](#Média-por-item)
       -  [Média dos itens por loja](#Média-dos-itens-por-loja)
   - [Lagged Variables](#Lagged-Variables) 
   
 > **Obs.:**
     - Vou criar primeiro features que são combinações de outras do dataset.
     - Depois vamos criar médias e somas com base em meses.
     - Por último entra as variáveis lagged, elas entram por último porque é importante manter a ordem delas.

### Revenue

A feature mais básica de todas é a receita (Dinheiro total ganho por item vendido).

In [95]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

### Somas mensais

In [96]:
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,0,59,22154,999.00,1.0,999.00
1,0,25,2552,899.00,1.0,899.00
2,0,25,2552,899.00,-1.0,-899.00
3,0,25,2554,1709.05,1.0,1709.05
4,0,25,2555,1099.00,1.0,1099.00


In [97]:
group_basic = ['date_block_num', 'shop_id', 'item_id']

# Não faz sentido somar o preço unitário de cada item.
df_train_month = train.drop(['item_price'], axis=1).groupby(group_basic).sum()

df_train_month.rename(columns={'item_cnt_day':'item_cnt_month', 'revenue':'revenue_month'}, inplace=True)
df_train_month.reset_index(inplace=True)

del group_basic

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month
0,0,0,32,6.0,1326.0
1,0,0,33,3.0,1041.0
2,0,0,35,1.0,247.0
3,0,0,43,1.0,221.0
4,0,0,51,2.0,257.0


### Médias mensais

Queremos tentar prever as vendas mensais do itens, então vou criar algumas features com base em médias mensais.

> **Obs.:**
   - Tirar médias do item_cnt_day não é muito preciso, porque algumas contém valores negativos. Deve ser pelo menos uma aproximação boa se não tivermos muitos números ímpares e valores pequenos.

### Média por loja

Vamos calcular a média do número de itens vendidos e da receita de cada loja por mês.

In [98]:
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,0,59,22154,999.00,1.0,999.00
1,0,25,2552,899.00,1.0,899.00
2,0,25,2552,899.00,-1.0,-899.00
3,0,25,2554,1709.05,1.0,1709.05
4,0,25,2555,1099.00,1.0,1099.00


In [99]:
aux = train.drop(['item_id', 'item_price', 'revenue'], axis=1)
aux = aux.groupby(['date_block_num', 'shop_id']).mean()
aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_shop'},
          inplace=True)

df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'shop_id'], how='left')

del aux

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop
0,0,0,32,6.0,1326.0,1.163781
1,0,0,33,3.0,1041.0,1.163781
2,0,0,35,1.0,247.0,1.163781
3,0,0,43,1.0,221.0,1.163781
4,0,0,51,2.0,257.0,1.163781


### Média por item

Vou calcular a média do número de itens vendidos e receita por item em cada mês.

In [100]:
aux = train.drop(['shop_id', 'item_price', 'revenue'], axis=1)
aux = aux.groupby(['date_block_num', 'item_id']).mean()

aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_item'},
          inplace=True)
df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'item_id'], how='left')

del aux

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item
0,0,0,32,6.0,1326.0,1.163781,1.328889
1,0,0,33,3.0,1041.0,1.163781,1.051724
2,0,0,35,1.0,247.0,1.163781,1.418182
3,0,0,43,1.0,221.0,1.163781,1.000000
4,0,0,51,2.0,257.0,1.163781,1.000000


### Média por categoria

Vai ser preciso unir as partes do dataset

In [101]:
train = pd.merge(train, items, on=['item_id'], how='left')
train = pd.merge(train, items_cat, on=['item_category_id'], how='left')
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,item_category_id,type_code,subtype_code
0,0,59,22154,999.00,1.0,999.00,37,11,1
1,0,25,2552,899.00,1.0,899.00,58,13,27
2,0,25,2552,899.00,-1.0,-899.00,58,13,27
3,0,25,2554,1709.05,1.0,1709.05,58,13,27
4,0,25,2555,1099.00,1.0,1099.00,56,13,3


In [102]:
df_train_month = pd.merge(df_train_month, items, on=['item_id'], how='left')
df_train_month = pd.merge(df_train_month, items_cat, on=['item_category_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8


In [103]:
aux = train.drop(['shop_id', 'item_id', 'type_code', 'subtype_code', 'item_price', 'revenue'], axis=1)
aux = aux.groupby(['date_block_num', 'item_category_id']).mean()

aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_category'},
          inplace=True)

df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'item_category_id'], how='left')

del aux

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4,1.145902
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1,1.053414
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4,1.145902
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4,1.145902
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8,1.003914


### Média por sub categoria

In [104]:
aux = train.drop(['shop_id', 'item_id', 'type_code', 'item_category_id', 'item_price', 'revenue'], axis=1)
aux = aux.groupby(['date_block_num', 'subtype_code']).mean()

aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_subtype_code'},
          inplace=True)

df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'subtype_code'], how='left')

del aux

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4,1.145902,1.145902
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1,1.053414,1.046046
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4,1.145902,1.145902
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4,1.145902,1.145902
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8,1.003914,1.003914


### Média por cidade

Vai ser preciso acabar de unificar o dataset

In [105]:
train = pd.merge(train, shops, on=['shop_id'], how='left')
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,item_category_id,type_code,subtype_code,city_code,latitude,longitude,is_online
0,0,59,22154,999.00,1.0,999.00,37,11,1,30,57.626076,39.884472,0
1,0,25,2552,899.00,1.0,899.00,58,13,27,13,55.755871,37.617680,0
2,0,25,2552,899.00,-1.0,-899.00,58,13,27,13,55.755871,37.617680,0
3,0,25,2554,1709.05,1.0,1709.05,58,13,27,13,55.755871,37.617680,0
4,0,25,2555,1099.00,1.0,1099.00,56,13,3,13,55.755871,37.617680,0


In [106]:
df_train_month = pd.merge(df_train_month, shops, on=['shop_id'], how='left')
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1,1.053414,1.046046,29,62.035454,129.675476,0
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8,1.003914,1.003914,29,62.035454,129.675476,0


In [107]:
aux = train[['date_block_num', 'city_code', 'item_cnt_day']]

aux = aux.groupby(['date_block_num', 'city_code']).mean()

aux.rename(columns={'item_cnt_day':'mean_item_cnt_per_city_code'},
          inplace=True)

df_train_month = pd.merge(df_train_month, aux, on=['date_block_num', 'city_code'], how='left')

del aux

df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online,mean_item_cnt_per_city_code
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1,1.053414,1.046046,29,62.035454,129.675476,0,1.13864
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8,1.003914,1.003914,29,62.035454,129.675476,0,1.13864


### Semestre e ano

A ideia aqui é criar colunas que indicam meses especiais com 1 e meses normais com 0. Por exemplo, podemos criar uma coluna na qual o mês de Dezembro é considerado especial (1). Isso deve ajudar o algoritmo a prever as vendas de natal.

In [108]:
# Criando uma coluna com zeros.
df_train_month['is_year'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
mask = (df_train_month.date_block_num + 1) % 12 == 0
df_train_month.loc[mask, 'is_year'] = 1

del mask

> <a name="back3"></a> **Obs.:** Se ficou muito complicado de entender o que aconteceu na última linha, eu coloquei um exemplo mais simples [aqui](#mask).

In [109]:
# Checando se os valores foram substituidos corretamente.
df_train_month[df_train_month.date_block_num == 11].head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online,mean_item_cnt_per_city_code,is_year
621448,11,2,33,1.0,199.0,1.136715,1.050000,37,11,1,1.049310,1.043893,0,44.48895,39.73082,0,1.136715,1
621449,11,2,482,1.0,3300.0,1.136715,1.128205,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1
621450,11,2,485,1.0,300.0,1.136715,1.191919,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1
621451,11,2,791,1.0,600.0,1.136715,1.031250,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1
621452,11,2,804,1.0,240.0,1.136715,1.040000,49,12,39,1.178918,1.178918,0,44.48895,39.73082,0,1.136715,1


In [110]:
# Criando uma coluna com zeros.
df_train_month['is_semester'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
mask = (df_train_month.date_block_num + 1) % 6 == 0
df_train_month.loc[mask, 'is_semester']  = 1

del mask

In [111]:
# Checando se os valores foram substituidos corretamente.
df_train_month[df_train_month.date_block_num == 11].head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online,mean_item_cnt_per_city_code,is_year,is_semester
621448,11,2,33,1.0,199.0,1.136715,1.050000,37,11,1,1.049310,1.043893,0,44.48895,39.73082,0,1.136715,1,1
621449,11,2,482,1.0,3300.0,1.136715,1.128205,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1,1
621450,11,2,485,1.0,300.0,1.136715,1.191919,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1,1
621451,11,2,791,1.0,600.0,1.136715,1.031250,73,15,0,1.108466,1.108466,0,44.48895,39.73082,0,1.136715,1,1
621452,11,2,804,1.0,240.0,1.136715,1.040000,49,12,39,1.178918,1.178918,0,44.48895,39.73082,0,1.136715,1,1


In [112]:
# Checando se os valores foram substituidos corretamente.
df_train_month[df_train_month.date_block_num == 5].head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online,mean_item_cnt_per_city_code,is_year,is_semester
295070,5,2,30,1.0,399.0,1.109333,1.484848,40,11,4,1.136362,1.136362,0,44.48895,39.73082,0,1.109333,0,1
295071,5,2,472,1.0,399.0,1.109333,1.333333,49,12,39,1.118893,1.118893,0,44.48895,39.73082,0,1.109333,0,1
295072,5,2,482,2.0,6600.0,1.109333,1.217391,73,15,0,1.097387,1.097387,0,44.48895,39.73082,0,1.109333,0,1
295073,5,2,484,6.0,1800.0,1.109333,1.444444,73,15,0,1.097387,1.097387,0,44.48895,39.73082,0,1.109333,0,1
295074,5,2,491,2.0,1200.0,1.109333,1.030303,73,15,0,1.097387,1.097387,0,44.48895,39.73082,0,1.109333,0,1


### Lagged Variables

Uma lagged feature é obtida ao atrasarmos ou adiantarmos a sequência daquela feature. Como por exemplo,

| Vendas_original  | Vendas_lag1    |
|----------|:------------- |
| 1 |  Nan |
| 2 |    1   |
| 3 | 2 |

A ideia por trás de fazermos essa transformação, é dar ao algoritmo a oportunidade de fazer escolhas como:
- O número de vendas desse mês é o mesmo número do mês passado.
- Dependendo do tamanho do lag é possível que o algoritmo aprenda sobre sazonalidade das vendas. Um bom exemplo é o natal, todo ano existe um aumento nas vendas neste período e o algoritmo pode levar isso em consideração.

<a name="lagged"></a> Deixei no apéndice uma explicação mais extensa sobre os [lagged variables](#lagged).

> Daqui pra frente as coisas ficam complicadas e eu não sei se vou utilizar lagged variables para treinar o modelo, mas já deixei o dataframe quase pronto para ser colocado em uma [6. LSTM](6.%20LSTM%20(Simples).ipynb).

In [113]:
import gc
gc.collect()

196

In [114]:
df_train_month.head()

,date_block_num,shop_id,item_id,item_cnt_month,revenue_month,mean_item_cnt_per_shop,mean_item_cnt_per_item,item_category_id,type_code,subtype_code,mean_item_cnt_per_category,mean_item_cnt_per_subtype_code,city_code,latitude,longitude,is_online,mean_item_cnt_per_city_code,is_year,is_semester
0,0,0,32,6.0,1326.0,1.163781,1.328889,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864,0,0
1,0,0,33,3.0,1041.0,1.163781,1.051724,37,11,1,1.053414,1.046046,29,62.035454,129.675476,0,1.13864,0,0
2,0,0,35,1.0,247.0,1.163781,1.418182,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864,0,0
3,0,0,43,1.0,221.0,1.163781,1.000000,40,11,4,1.145902,1.145902,29,62.035454,129.675476,0,1.13864,0,0
4,0,0,51,2.0,257.0,1.163781,1.000000,57,13,8,1.003914,1.003914,29,62.035454,129.675476,0,1.13864,0,0


Vamos diminuir o tamanho do dataset trocando o tipo de suas variáveis.

In [115]:
df_train_month['date_block_num'] = df_train_month['date_block_num'].astype(np.int8)
df_train_month['shop_id'] = df_train_month['shop_id'].astype(np.int8)
df_train_month['item_id'] = df_train_month['item_id'].astype(np.int16)
df_train_month['item_cnt_month'] = df_train_month['item_cnt_month'].astype(np.int16)
df_train_month['revenue_month'] = df_train_month['revenue_month'].astype(np.float32)

df_train_month['mean_item_cnt_per_shop'] = df_train_month['mean_item_cnt_per_shop'].astype(np.float32)
df_train_month['mean_item_cnt_per_item'] = df_train_month['mean_item_cnt_per_item'].astype(np.float32)
df_train_month['mean_item_cnt_per_category'] = df_train_month['mean_item_cnt_per_category'].astype(np.float32)
df_train_month['mean_item_cnt_per_subtype_code'] = df_train_month['mean_item_cnt_per_subtype_code'].astype(np.float32)
df_train_month['mean_item_cnt_per_city_code'] = df_train_month['mean_item_cnt_per_city_code'].astype(np.float32)

df_train_month['item_category_id'] = df_train_month['item_category_id'].astype(np.int16)
df_train_month['type_code'] = df_train_month['type_code'].astype(np.int16)
df_train_month['subtype_code'] = df_train_month['subtype_code'].astype(np.int16)

df_train_month['is_year'] = df_train_month['is_year'].astype(np.int8)
df_train_month['is_semester'] = df_train_month['is_semester'].astype(np.int8)
df_train_month['city_code'] = df_train_month['city_code'].astype(np.int16)
df_train_month['latitude'] = df_train_month['latitude'].astype(np.float32)
df_train_month['longitude'] = df_train_month['longitude'].astype(np.float32)
df_train_month['is_online'] = df_train_month['is_online'].astype(np.int8)

Salvando os tipos de cada coluna, vamos precisar mais pra frente.

In [116]:
column_type_dict = {}

for var in zip(df_train_month.columns, df_train_month.dtypes):
    column_type_dict[var[0]] = var[1]

In [117]:
# Tudo que não se modifica com o passar dos meses entra no index.
index = ['shop_id', 'item_id']# 'item_category_id', 'type_code', 'subtype_code',
         #'is_year', 'is_semester', 'city_code',
         #'latitude', 'longitude', 'is_online']

# Tudo que se modifica com o passar dos meses entra no valores e vão virar lagged variables.
values = ['item_cnt_month', 'revenue_month',
          'mean_item_cnt_per_shop', 'mean_item_cnt_per_item',
          'mean_item_cnt_per_category', 'mean_item_cnt_per_subtype_code',
          'mean_item_cnt_per_city_code']

columns = ['date_block_num']

In [118]:
df_train_month = df_train_month.pivot_table(index=index, columns=columns, values=values, fill_value=0)

In [121]:
for key in values:
    df_train_month[key] = df_train_month[key].astype(column_type_dict[key])

In [125]:
first_lvl_columns = values[:2]#df_train_month.columns.levels[0]
lags = np.arange(6) + 1

for column in first_lvl_columns:
    
    for lag in lags:
        
        df_aux_shifted = df_train_month[column].shift(lag, axis=1)
        
        # Faltou habilidade aqui.
        df_aux_shifted = pd.concat([df_aux_shifted], keys=[column + '_lag(' + str(-lag) + ')'], axis=1)
        
        df_train_month = pd.concat([df_train_month, df_aux_shifted], join='outer', axis=1)

    df_train_month.rename(columns={column: column + '_lag(' + str(0) + ')'}, inplace=True)

del df_aux_shifted

In [127]:
col_ordered = []

for column in first_lvl_columns:
    
    for lag in range(lags[-1], -1, -1):
        col_ordered.append(column + '_lag(' + str(-lag) + ')')

In [ ]:
df_train_month = df_train_month.stack()

In [82]:
df_train_month[col_ordered][0:10]

item_cnt_month_lag(-6)  \
shop_id item_id date_block_num                           
0       30      0                                  NaN   
                1                                  NaN   
                2                                  NaN   
                3                                  NaN   
                4                                  NaN   
                5                                  NaN   
                6                                  0.0   
                7                                 31.0   
                8                                  0.0   
                9                                  0.0   

                                item_cnt_month_lag(-5)  \
shop_id item_id date_block_num                           
0       30      0                                  NaN   
                1                                  NaN   
                2                                  NaN   
                3                                  NaN   
                4                                  NaN   
                5                                  0.0   
                6                                 31.0   
                7                                  0.0   
                8                                  0.0   
                9                                  0.0   

                                item_cnt_month_lag(-4)  \
shop_id item_id date_block_num                           
0       30      0                                  NaN   
                1                                  NaN   
                2                                  NaN   
                3                                  NaN   
                4                                  0.0   
                5                                 31.0   
                6                                  0.0   
                7                                  0.0   
                8                                  0.0   
                9                                  0.0   

                                item_cnt_month_lag(-3)  \
shop_id item_id date_block_num                           
0       30      0                                  NaN   
                1                                  NaN   
                2                                  NaN   
                3                                  0.0   
                4                                 31.0   
                5                                  0.0   
                6                                  0.0   
                7                                  0.0   
                8                                  0.0   
                9                                  0.0   

                                item_cnt_month_lag(-2)  \
shop_id item_id date_block_num                           
0       30      0                                  NaN   
                1                                  NaN   
                2                                  0.0   
                3                                 31.0   
                4                                  0.0   
                5                                  0.0   
                6                                  0.0   
                7                                  0.0   
                8                                  0.0   
                9                                  0.0   

                                item_cnt_month_lag(-1)  item_cnt_month_lag(0)  \
shop_id item_id date_block_num                                                  
0       30      0                                  NaN                      0   
                1                                  0.0                     31   
                2                                 31.0                      0   
                3                                  0.0                      0   
                4                                  0.0       

In [ ]:
#df_train_month.head()

In [ ]:
df_train_month = df_train_month.reset_index()

df_train_month = pd.merge(df_train_month, items, on=['item_id'], how='left')
df_train_month = pd.merge(df_train_month, items_cat, on=['item_category_id'], how='left')
df_train_month = pd.merge(df_train_month, shops, on=['shop_id'], how='left')

df_train_month.head()

In [ ]:
df_train_month.dropna(axis=0, inplace=True)
df_train_month.head()

In [ ]:
for column in df_train_month.columns:
    
    name = column.split('_')
    
    if name[-1][:3] == 'lag':
        
        name = "_".join(name[:-1])
        
        df_train_month[column] = df_train_month[column].astype(column_type_dict[name])
        
    else:
        
        df_train_month[column] = df_train_month[column].astype(column_type_dict[column])

In [ ]:
df_train_month.info()

In [ ]:
import pickle
df_train_month.to_pickle('df_train_month_lag12.pkl')

In [ ]:
del train
del shops
del items
del items_Cat
del df_train_month

gc.collect()

-----------------

# Apêndice

### Trocando features categóricas por números ou vetores

<a name="go1"></a>No código abaixo, x vai guardar a relações entre os rótulos,
-  Ex.: a = 1, b = 2, e etc.

Isso é importante se quisermos fazer a transformação reversa.

[Voltar ao notebook](#back1)

In [ ]:
x = LabelEncoder()

vetor_normal = ['a','b','c','d']

vetor_transformado = x.fit_transform(vetor_normal)

print('Categorias', vetor_normal)
print('Rótulos:', vetor_transformado)

voltando = x.inverse_transform(vetor_transformado)

print('Invertendo a transformação:', voltando)

One-hot enconding.

In [ ]:
one = OneHotEncoder(handle_unknown='ignore')
vetor = [['Laranja', 'bom'], ['Maçã', 'bom'], ['Cebola', 'ruim']]
one.fit(vetor)

In [ ]:
one_hot = one.transform(vetor).toarray()
print('Representação em forma de vetor:')
print(['Laranja', 'bom'], '=', one_hot[0])
print(['Maçã', 'bom'], '=', one_hot[1])
print(['Cebola', 'ruim'], '=', one_hot[2])

Quando colocamos um vetor que não existe, como ['Chocolate, 'médio'], e o handle_unknown='ignore', a função vai retornar um vetor $0$. Agora, se colocarmos um vetor que não estava presente inicialmente, mas é uma combinação dos outros como ['Cebola', 'bom'], então conseguimos uma representação vetorial.

In [ ]:
one_hot = one.transform([['Chocolate', 'médio'], ['Cebola', 'bom']]).toarray()
print('Não estava presente nas categórias:')
print(['Chocolate', 'médio'], '=', one_hot[0])
print('Combinação de outras categórias:')
print(['Cebola', 'bom'], '=', one_hot[1])

----------------------------------------

### <a name="lagged"></a>  Lagged um pouco mais explicada

[Voltar ao notebook](#back2)

In [ ]:
a = pd.merge(a, a.shift(-1, axis=1), left_index=True, right_index=True, how='inner', suffixes=('_-1', '_0'))

Explicando lagged variables

In [ ]:
# Criando um pandas dataframe com o número de vendas de 1 até 6.
df_aux = pd.DataFrame(np.linspace(1,6,6), columns=['Vendas'])
df_aux

> **Obs.:** df_aux simula um dataset das vendas de uma loja, onde a passagem de tempo seria algo do tipo:
- linha 0 = mês de Janeiro.
- linha 1 = mês de Fevereiro.

In [ ]:
#Usando a função shift para deslocar o array em 1.
df_aux['Vendas_shift_+'] = df_aux['Vendas'].shift(1)
df_aux['Vendas_shift_-'] = df_aux['Vendas'].shift(-1)

df_aux

Temos que descartar as linhas que possuem Nan's.

In [ ]:
df_aux.dropna(inplace=True)
df_aux

Note que as colunas estão fora de ordem, para que o algoritmo reconheça a passagem de tempo é necessário reorganziar as colunas.

In [ ]:
df_aux = df_aux[['Vendas_shift_+', 'Vendas', 'Vendas_shift_-']]
df_aux

É mais conveninente renomear as colunas tmb.

In [ ]:
df_aux.rename(columns={'Vendas_shift_+': 'Vendas_t-2',
                       'Vendas': 'Vendas_t-1',
                       'Vendas_shift_-': 'Vendas_t'}
              , inplace=True)
df_aux

Compare essa tabela com a gerada inicialmente, onde a passagem dos meses seguiam o número das linhas. Na nova tabela, a passagem de tempo pode ser vista seguindo as colunas $t-2$, $t-1$, $t$.

> **Obs.:** Esse tipo de organização é perfeita para usarmos em algoritmo de Machine Learning. As linhas seriam suas amostras, as features seriam $t-2$ e $t-1$, e o que você estaria prevendo seria $t$.

### Código para gerar e organizar as variáveis com lag

O processo de criar lagged features é bem chato se for feito manualmente igual fizemos acima, principalmente se fizermos com muitas variáveis. Acabei fazendo um função pra fazer isso. Ela foi baseado em outra função encontrada [aqui][lag] (ótimo site para aprender Machine Learning).

[lag]: https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/

> Acabei não usando essa função nesse notebook, mas ela é bem útil para gerar lagged variables.

In [ ]:
def time_shift(df, lags, dropnan=True):
    
    '''
    Adiciona novas colunas deslocadas no tempo.
    Argumentos:
    
        df: Pandas DataFrame.
        
        lags: lista com o número de passos
               a ser deslocado.
               
        dropnan: (True) Retira as linhas que
                teham Nan's.
                
    Retorna:
        Pandas DataFrame com as colunas deslocadas.
    '''
    lags.append(0)
    lags = -1 * np.sort(lags)
    
    columns = [str(df.name)] if type(df) is pd.Series else df.columns
    col = []
    
    for column in columns:
        for lag in lags: 
            
            if lag == 0:
                new_column = column 
                col.append(new_column)
            else:
                new_column = column + '_lag(' + str(-1 * lag) + ')'
                col.append(new_column)
                df[new_column] = df[column].shift(lag)
                
    if dropnan:
        df.dropna(inplace=True)
        
    return df[col]

### <a name="mask"></a> Exemplo para seleção do mês ano

[Voltar ao notebook](#back3)

Vamos criar um array com números de 1 a 36.

In [ ]:
month = np.linspace(1,36,36)
month

Ao fazermos (month % 12), o python retorna o resto da divisão do array month por 12, ou seja, os únicos números que divisíveis são 12, 24, 36.

In [ ]:
(month % 12)

Agora, ao adicionarmos uma comparação (month % 12 == 0), queremos saber em quais posições do arrya a condição (== 0) é verdade.

In [ ]:
(month % 12 == 0)

Logo, ao colocarmos esse tipo de array no pandas queremos dizer: Só faça o que pedimos nas posições onde temos **True**, não faça nada se for **False**.

----------------------------------------

# Referências
1. Muitas das features craidas neste notebook foram retiradas deste [kernel](https://www.kaggle.com/dlarionov/feature-engineering-xgboost).
2. Site de onde tirei as informações sobre lagged variables. [Machine Learning Mastery](https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/)
3. Site para encontrar as latitudes e longitudes. [Latlong](https://www.latlong.net/).